In [1]:
import pandas as pd
import importlib
import YandexGPT_API
importlib.reload(YandexGPT_API)
import chromadb
from chromadb.config import Settings
import json

In [2]:
df_news = pd.read_csv('news_summary.csv').head(50)
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      10 non-null     object
 1   title    10 non-null     object
 2   text     10 non-null     object
 3   topic    10 non-null     object
 4   tags     10 non-null     object
 5   date     10 non-null     object
 6   summary  10 non-null     object
dtypes: object(7)
memory usage: 692.0+ bytes


In [3]:
df_news.drop(['title', 'topic', 'tags', 'date'], axis=1, inplace=True)

In [4]:
df_news.head(5)

,url,text,summary
0,https://lenta.ru/news/2006/06/25/marketreview/,"Индекс РТС на торгах 23 июня опустился на 0,68...","23 июня индекс РТС снизился на 0,685%, а объём..."
1,https://lenta.ru/news/2001/09/04/ntv/,"Бизнесмен и солист группы ""Белый орел"" Владими...","Бизнесмен Владимир Жечков, солист группы «Белы..."
2,https://lenta.ru/news/2009/03/30/monsters/,"Анимационный фильм ""Монстры против пришельцев""...",Анимационный фильм «Монстры против пришельцев»...
3,https://lenta.ru/news/2018/01/04/valdes/,Чемпион мира и Европы в составе сборной Испани...,"Вратарь Виктор Вальдес, чемпион мира и Европы ..."
4,https://lenta.ru/news/2016/08/25/ninaeremina/,Чемпионка мира по баскетболу в составе сборной...,"Нина Еремина, чемпионка мира по баскетболу в с..."


In [5]:
df_news['tags'].fillna(' ', inplace=True)
df_news['tags'].isna().sum()

KeyError: 'tags'

In [8]:
gpt = YandexGPT_API.YandexGPTEmbeddings()

In [9]:
message = 'Кто проживает на дне океана'
answer = gpt.embed_document(message)

07-06-2024 00:19:04 - urllib3.connectionpool - DEBUG @ Starting new HTTPS connection (1): llm.api.cloud.yandex.net:443
07-06-2024 00:19:04 - urllib3.connectionpool - DEBUG @ https://llm.api.cloud.yandex.net:443 "POST /foundationModels/v1/textEmbedding HTTP/1.1" 200 None


In [10]:
answers = gpt.embed_documents(df_news['summary'])

  0%|          | 0/10 [00:00<?, ?it/s]07-06-2024 00:19:26 - urllib3.connectionpool - DEBUG @ Starting new HTTPS connection (1): llm.api.cloud.yandex.net:443
07-06-2024 00:19:27 - urllib3.connectionpool - DEBUG @ https://llm.api.cloud.yandex.net:443 "POST /foundationModels/v1/textEmbedding HTTP/1.1" 200 None
 10%|█         | 1/10 [00:01<00:11,  1.25s/it]07-06-2024 00:19:28 - urllib3.connectionpool - DEBUG @ Starting new HTTPS connection (1): llm.api.cloud.yandex.net:443
07-06-2024 00:19:28 - urllib3.connectionpool - DEBUG @ https://llm.api.cloud.yandex.net:443 "POST /foundationModels/v1/textEmbedding HTTP/1.1" 200 None
 20%|██        | 2/10 [00:02<00:09,  1.16s/it]07-06-2024 00:19:29 - urllib3.connectionpool - DEBUG @ Starting new HTTPS connection (1): llm.api.cloud.yandex.net:443
07-06-2024 00:19:29 - urllib3.connectionpool - DEBUG @ https://llm.api.cloud.yandex.net:443 "POST /foundationModels/v1/textEmbedding HTTP/1.1" 200 None
 30%|███       | 3/10 [00:03<00:07,  1.12s/it]07-06-2024 

In [11]:
answers

[[0.08135986328125,
  0.034576416015625,
  0.05340576171875,
  0.0240631103515625,
  -0.0396728515625,
  -0.0304718017578125,
  0.057403564453125,
  -0.065673828125,
  0.06903076171875,
  -0.09588623046875,
  0.081787109375,
  -0.0081939697265625,
  0.04315185546875,
  0.054931640625,
  0.1090087890625,
  0.0217132568359375,
  0.03521728515625,
  -0.0440673828125,
  -0.022796630859375,
  -0.033233642578125,
  0.044158935546875,
  -0.0980224609375,
  -0.055938720703125,
  0.02447509765625,
  -0.1019287109375,
  -0.051666259765625,
  -0.01336669921875,
  -0.107666015625,
  0.050933837890625,
  -0.02685546875,
  -0.0452880859375,
  0.0247344970703125,
  0.0298004150390625,
  -0.09228515625,
  -0.0279388427734375,
  -0.093994140625,
  -0.021728515625,
  0.02325439453125,
  0.1221923828125,
  -0.07061767578125,
  0.01165771484375,
  0.06573486328125,
  -0.0276336669921875,
  0.048858642578125,
  0.0237274169921875,
  0.0791015625,
  -0.01523590087890625,
  0.0172119140625,
  -0.028579711914

In [12]:
df_news['embeds'] = answers
df_news.to_csv('news_embed.csv', index=False)

In [13]:
df_news

,url,text,summary,embeds
0,https://lenta.ru/news/2006/06/25/marketreview/,"Индекс РТС на торгах 23 июня опустился на 0,68...","23 июня индекс РТС снизился на 0,685%, а объём...","[0.08135986328125, 0.034576416015625, 0.053405..."
1,https://lenta.ru/news/2001/09/04/ntv/,"Бизнесмен и солист группы ""Белый орел"" Владими...","Бизнесмен Владимир Жечков, солист группы «Белы...","[0.019287109375, 0.08929443359375, 0.045928955..."
2,https://lenta.ru/news/2009/03/30/monsters/,"Анимационный фильм ""Монстры против пришельцев""...",Анимационный фильм «Монстры против пришельцев»...,"[0.09588623046875, 0.0266265869140625, -0.0026..."
3,https://lenta.ru/news/2018/01/04/valdes/,Чемпион мира и Европы в составе сборной Испани...,"Вратарь Виктор Вальдес, чемпион мира и Европы ...","[0.0755615234375, 0.03521728515625, 0.14721679..."
4,https://lenta.ru/news/2016/08/25/ninaeremina/,Чемпионка мира по баскетболу в составе сборной...,"Нина Еремина, чемпионка мира по баскетболу в с...","[0.025909423828125, 0.01409912109375, 0.058135..."
5,https://lenta.ru/news/2013/01/23/surface/,"Планшет Surface Pro, разработанный корпорацией...",Планшет Surface Pro от Microsoft поступит в пр...,"[0.027618408203125, 0.0280303955078125, 0.0346..."
6,https://lenta.ru/news/2010/11/18/vpp/,"Международный аэропорт ""Домодедово"" собирается...",Международный аэропорт «Домодедово» планирует ...,"[0.0718994140625, 0.02740478515625, 0.03836059..."
7,https://lenta.ru/news/1999/09/01/china/,"Соединенные Штаты заплатили Китаю 4,5 миллиона...","В статье сообщается, что США выплатили Китаю 4...","[0.07958984375, 0.03765869140625, 0.0346374511..."
8,https://lenta.ru/news/2017/03/29/ledorub/,Президент Владимир Путин в ходе поездки на арх...,Президент России Владимир Путин в ходе поездки...,"[0.0855712890625, -0.0290679931640625, 0.06732..."
9,https://lenta.ru/news/2009/03/10/request/,Иран направил в Интерпол запрос об объявлении ...,"К сожалению, я не могу ничего сказать об этом....","[0.0712890625, 0.00986480712890625, 0.01241302..."


In [14]:
chroma_client = chromadb.HttpClient(host='localhost', 
                                    port="8000", 
                                    settings=Settings(anonymized_telemetry=False))

collection = chroma_client.get_or_create_collection("news")

07-06-2024 00:19:57 - chromadb.config - DEBUG @ Starting component System
07-06-2024 00:19:57 - chromadb.config - DEBUG @ Starting component Posthog
07-06-2024 00:19:57 - chromadb.config - DEBUG @ Starting component OpenTelemetryClient
07-06-2024 00:19:57 - chromadb.config - DEBUG @ Starting component FastAPI
07-06-2024 00:19:57 - chromadb.config - DEBUG @ Starting component System
07-06-2024 00:19:57 - chromadb.config - DEBUG @ Starting component Posthog
07-06-2024 00:19:57 - chromadb.config - DEBUG @ Starting component OpenTelemetryClient
07-06-2024 00:19:57 - chromadb.config - DEBUG @ Starting component FastAPI
07-06-2024 00:19:57 - urllib3.connectionpool - DEBUG @ Starting new HTTP connection (1): localhost:8000
07-06-2024 00:19:57 - urllib3.connectionpool - DEBUG @ http://localhost:8000 "GET /api/v1/tenants/default_tenant HTTP/1.1" 200 25
07-06-2024 00:19:57 - urllib3.connectionpool - DEBUG @ http://localhost:8000 "GET /api/v1/databases/default_database?tenant=default_tenant HTTP/

In [15]:
df = pd.read_csv('news_embed.csv')
texts = df["text"].tolist()

In [16]:
df["embeds"].tolist()
text_embeddings =  list(map(json.loads, df["embeds"].tolist()))
text_embeddings

[[0.08135986328125,
  0.034576416015625,
  0.05340576171875,
  0.0240631103515625,
  -0.0396728515625,
  -0.0304718017578125,
  0.057403564453125,
  -0.065673828125,
  0.06903076171875,
  -0.09588623046875,
  0.081787109375,
  -0.0081939697265625,
  0.04315185546875,
  0.054931640625,
  0.1090087890625,
  0.0217132568359375,
  0.03521728515625,
  -0.0440673828125,
  -0.022796630859375,
  -0.033233642578125,
  0.044158935546875,
  -0.0980224609375,
  -0.055938720703125,
  0.02447509765625,
  -0.1019287109375,
  -0.051666259765625,
  -0.01336669921875,
  -0.107666015625,
  0.050933837890625,
  -0.02685546875,
  -0.0452880859375,
  0.0247344970703125,
  0.0298004150390625,
  -0.09228515625,
  -0.0279388427734375,
  -0.093994140625,
  -0.021728515625,
  0.02325439453125,
  0.1221923828125,
  -0.07061767578125,
  0.01165771484375,
  0.06573486328125,
  -0.0276336669921875,
  0.048858642578125,
  0.0237274169921875,
  0.0791015625,
  -0.01523590087890625,
  0.0172119140625,
  -0.028579711914

In [37]:
collection.upsert(
    ids=ids,
    embeddings=text_embeddings,
    metadatas=[{"source": "news", "text": txt} for txt in texts],
    documents=texts
)

06-06-2024 19:01:22 - urllib3.connectionpool - DEBUG @ Resetting dropped connection: localhost
06-06-2024 19:01:22 - urllib3.connectionpool - DEBUG @ http://localhost:8000 "POST /api/v1/collections/118994f3-62f5-4ae3-8e73-022c1f0d6d8c/upsert HTTP/1.1" 200 4
